# Importaciones

In [1]:
from imports import *                    # pandas, numpy, etc.
from trading_environment import (
    DataManager,
    Optimizer,
    PerformanceAnalyzer,
    ReportManager,
    RiskManager,
    DATA_DIR,
    MAX_FILE_SIZE,
    REPORT_DIR
)
from strategies import (
    MovingAverageCrossStrategy,
    DcaStrategy,
    DonchianBreakoutStrategy,
    ADXTrendStrategy,
    ROCStrategy,
    MACDStrategy,
    BollingerMeanRevStrategy,
    RSIStrategy,
    PairTradingStrategy,
    VWAPStrategy
)

# Strategies

In [2]:
# 1) Load data
dm = DataManager(data_dir=DATA_DIR, max_file_size=MAX_FILE_SIZE)
data = dm.load_data(
    symbols='BTC-USD',
    period='2y',
    interval='1d'
)

2025-06-26 14:41:10,657 - INFO - Cargados 731 registros de caché (1 archivos)


In [3]:
strategies = [
    MovingAverageCrossStrategy, DcaStrategy, DonchianBreakoutStrategy,
    ADXTrendStrategy, ROCStrategy, MACDStrategy, BollingerMeanRevStrategy,
    RSIStrategy, PairTradingStrategy, VWAPStrategy
]

# run Bayesian optimisation on each and pick the best overall by Sharpe:
best_strategy, best_params, summary_df = Optimizer.find_best_strategy(
    strategies,
    data,
    method='bayes',
    metric='Sharpe',
    n_trials=50,       # passed into optimize_bayesian
    seed=42
)



[I 2025-06-26 14:41:10,666] A new study created in memory with name: no-name-4eb4a87c-104d-4a20-a0f2-860cfc23c049
[W 2025-06-26 14:41:10,672] Trial 0 failed with parameters: {'short_window': 40, 'long_window': 200} because of the following error: The value nan is not acceptable.
[W 2025-06-26 14:41:10,673] Trial 0 failed with value nan.
[W 2025-06-26 14:41:10,680] Trial 1 failed with parameters: {'short_window': 75, 'long_window': 130} because of the following error: The value nan is not acceptable.
[W 2025-06-26 14:41:10,681] Trial 1 failed with value nan.
[W 2025-06-26 14:41:10,686] Trial 2 failed with parameters: {'short_window': 20, 'long_window': 40} because of the following error: The value nan is not acceptable.
[W 2025-06-26 14:41:10,686] Trial 2 failed with value nan.
[W 2025-06-26 14:41:10,691] Trial 3 failed with parameters: {'short_window': 10, 'long_window': 180} because of the following error: The value nan is not acceptable.
[W 2025-06-26 14:41:10,692] Trial 3 failed wit

ValueError: No trials are completed yet.

In [ ]:
print("Best strategy:", best_strategy)
print("Params:       ", best_params)

In [ ]:
summary_df

# FIN